In [716]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [717]:
from deluca.agents import DRC, LQR
from deluca.envs import LDS
import jax.numpy as jnp

In [718]:
A,B = jnp.array([[.8,.5], [0,.8]]), jnp.array([[0],[0.8]])
lds = LDS(state_size= B.shape[0], action_size=B.shape[1], A=A, B=B)

In [719]:
drc = DRC(lds.A, lds.B)
drc_lqr = DRC(lds.A, lds.B, K= LQR(A, B).K)
lqr = LQR(A, B)

In [720]:
def get_err(T, lds, controller):
    lds.reset()
    avg_err = 0
    for i in range(T):
        action = controller(lds.state)
        lds.step(action)
        lds.state += 0.5 * jnp.sin(i) # add sine noise
        avg_err += (jnp.linalg.norm(lds.state)+jnp.linalg.norm(action))/T
    return avg_err

In [721]:
T = 100

In [722]:
print("Pure DRC incurs ", get_err(T, lds, drc), " loss")
print("DRC initialized with LQR incurs ", get_err(T, lds, drc_lqr), " loss")
print("LQR incurs ", get_err(T, lds, lqr), " loss")

Pure DRC incurs  0.558123  loss
DRC initialized with LQR incurs  0.8183247  loss
LQR incurs  1.2538122  loss
